## 2. Create Required Azure Services
You should run first jupyter notebook (01_setup_environment.ipynb) in this sample to set the global variables. Once it is run, you dont need to repeat it again and again as the variables etc. stored in an environment file. You may need to run it again only in case you delete the ".env" file or your Azure login session expires.

### 2.1. Get global variables
We will read the previously stored variables

In [30]:
from dotenv import set_key, get_key, find_dotenv
envPath = "/data/home/mahesh/TF/curbside/notebooks/.env"

azureSubsctiptionId = get_key(envPath, "azureSubsctiptionId")
resourceLocation = get_key(envPath, "resourceLocation")
resourceGroupName = get_key(envPath, "resourceGroupName")
containerRegServiceName = get_key(envPath, "containerRegServiceName")
iotHubServiceName = get_key(envPath, "iotHubServiceName")
iotDeviceId = get_key(envPath, "iotDeviceId")

### 2.2. Create Resource Group

In [ ]:
!az group create --name $resourceGroupName --location  $resourceLocation

### 2.3. Create Azure Container Registry Service

In [ ]:
!az acr create --name $containerRegServiceName --resource-group $resourceGroupName --location $resourceLocation --sku Basic --admin-enabled true

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
Even the execution of above cell seems to be finished, it may take some time (several seconds) to have the changes reflected on Azure Datacenter. So if you immediately run the below cells after the execution of above one, it may give error by complaining that the ACR service doesnt exists...


#### 2.3.1. Get access credentials to the ACR service

In [31]:
%%bash --out output -s "$containerRegServiceName"
az acr credential show -n $1

In [32]:
import json
jsonRegDetails = json.loads(output)

acrUserName = jsonRegDetails['username']
acrPassword = jsonRegDetails['passwords'][0]['value']

Now save these information into the **.env** file for further use.

In [33]:
set_key(envPath, "acrUserName", acrUserName)
set_key(envPath, "acrPassword", acrPassword)
set_key(envPath, "acrServer", containerRegServiceName+".azurecr.io")

(True, 'acrServer', 'teamlvacontainerregistry.azurecr.io')

### 2.4. Create Azure IoT Hub
Below cell may give following error: "IotHub name '<NAME\>' is not available" in case service already exist.


In [ ]:
!az iot hub create --name $iotHubServiceName --resource-group $resourceGroupName --sku S1

### 2.5. Create IoT Edge Device
Create an IoT Edge device under the Azure IoT Hub service created in above cells. In the further steps, when we connect / match the physical device with this cloud based one through the device connection string, everything in this cloud one will be cloned into the physical device (all IoT Edge modules etc.)

In this step we are also printing out and saving the IoT Edge device connection string which will be used in !!!further steps!!!.

In [6]:
# Install iot hub CLI extensions
!az extension add --name azure-cli-iot-ext

- Installing ...

In [34]:
%%bash --out iotEdgeDeviceConnString -s "$resourceGroupName" "$iotHubServiceName" "$iotDeviceId"
# Create edge device if not already exists...
az iot hub show --resource-group $1 --name $2 &> /dev/null

if [ $? -ne 0 ]; then
    echo "somethin happened"
    #az iot hub create --hub-name $2 --device-id $3 --edge-enabled &> /dev/null
fi

az iot hub show-connection-string --name $2 --output tsv

In [35]:
# Print out IoT Edge Device deteails and store its connection string...
print('Azure IoT Hub Name: {}'.format(iotHubServiceName))
print('Azure IoT device id : {}'.format(iotDeviceId))
print('IoT Edge Device Connection String: {}'.format(iotEdgeDeviceConnString))

set_key(envPath, "iotEdgeDeviceConnString", iotEdgeDeviceConnString.rstrip())

Azure IoT Hub Name: teamlvahub
Azure IoT device id : teamlvadbe-edge
IoT Edge Device Connection String: HostName=teamlvahub.azure-devices.net;SharedAccessKeyName=iothubowner;SharedAccessKey=ItKv4k08OF6e2wnjqPPMhjLBe/OSBH/rkDoZ05y5qIg=



(True,
 'iotEdgeDeviceConnString',
 'HostName=teamlvahub.azure-devices.net;SharedAccessKeyName=iothubowner;SharedAccessKey=ItKv4k08OF6e2wnjqPPMhjLBe/OSBH/rkDoZ05y5qIg=')